# Calculating Overlap in Actual Imagery Coverage Between Baseline and EV Imagery Area 

In [ ]:
import pandas as pd
import geopandas as gpd
import sys
sys.path.append('../src/')
import image_coverage as img_cover
import analyze_img_coverage as analyze
from datetime import datetime, timedelta

### Set File Paths

In [ ]:
# List of approved projects (has country codes)
approved_projects = '../projects_all_approved_202502211226.csv'

# Polygon geometries and attributes from TM API
polygon_features = '../data/BEF_shortlist/tm_api_BEF_shortlist_2025-06-02.csv'

# All Maxar imagery & image footprints for polygons
maxar_md = '../data/BEF_shortlist/imagery_availability/comb_img_availability_BEF_shortlist_2025-06-02.csv'

# Best image availability per polygon
# Baseline
base_img_avail = '../data/BEF_shortlist/results/baseline/polygon_imagery_coverage_BEF_shortlist_baseline_2025-06-02.csv'

# Early Verification (1+ year post-plantstart)
ev_img_avail = '../data/BEF_shortlist/results/ev/polygon_imagery_coverage_BEF_shortlist_ev_2025-06-02.csv'

### Read in Files

In [ ]:
# List of approved projects (has country codes)
approved_projects_df = pd.read_csv(approved_projects)

# Polygon geometries and attributes from TM API
poly_df = pd.read_csv(polygon_features)

# All Maxar imagery & image footprints for polygons
maxar_df = pd.read_csv(maxar_md)

# Best image availability per polygon
# Baseline
base_img_avail_df = pd.read_csv(base_img_avail)

# Early Verification (1+ year post-plantstart)
ev_img_avail_df = pd.read_csv(ev_img_avail)

### Preprocess Polygon & Maxar Data

In [ ]:
poly_gdf = img_cover.preprocess_polygons(poly_df, debug=True)
maxar_gdf = img_cover.preprocess_images(maxar_df, debug=True)

## Build Comparison Dataframe

#### Baseline Polygon Image Availability

In [ ]:
# Merge in 'poly_geom' from poly_gdf to baseline image availability df
base_img_avail_gdf = gpd.GeoDataFrame(base_img_avail_df.merge(poly_gdf[['poly_id', 'poly_geom']], how='left', on='poly_id'))

In [ ]:
# Create lookup table of image geometry associated with each unique maxar image ID
unique_maxar_imgs_gdf = maxar_gdf.drop_duplicates(subset='title')[['title', 'img_geom']]
unique_maxar_imgs_gdf.head(2)

# Merge in 'img_geom' from maxar_gdf lookup table to baseline image availability df
base_img_avail_gdf = base_img_avail_gdf.merge(unique_maxar_imgs_gdf[['title', 'img_geom']], how='left', left_on='best_image', right_on='title')
base_img_avail_gdf[117:120]

#### Early Verification Polygon Image Availability

In [ ]:
# Merge in 'poly_geom' from poly_gdf to early verification image availability df
ev_img_avail_gdf = gpd.GeoDataFrame(ev_img_avail_df.merge(unique_maxar_imgs_gdf[['title', 'img_geom']], how='left', left_on='best_image', right_on='title'))
ev_img_avail_gdf.head(2)

#### Combine Baseline & Early Verification Image Availability GeoDataFrames

In [ ]:
#Select and rename relevent columns from baseline dataframe
combined_gdf = base_img_avail_gdf[[
    'poly_id', 'project_id', 'poly_geom', 'poly_area_ha', 'best_image', 'img_geom', 'overlap_area_ha', 'percent_img_cover']].copy().rename(columns={
    'best_image': 'best_img_base',
    'img_geom': 'img_geom_base',
    'overlap_area_ha': 'overlap_area_base',
    'percent_img_cover': 'pct_img_cover_base'
})

In [ ]:
# Merge in 'best_image' and 'img_geom' from the early verification dataframe
combined_gdf = combined_gdf.merge(ev_img_avail_gdf[[
    'poly_id', 'best_image', 'img_geom', 'overlap_area_ha', 'percent_img_cover']], how='left', on='poly_id').rename(
    columns={
        'best_image': 'best_img_ev',
        'img_geom': 'img_geom_ev',
        'overlap_area_ha': 'overlap_area_ev',
        'percent_img_cover': 'pct_img_cover_ev'
    })

In [ ]:
print('length of gdf', len(combined_gdf))
print('poly_geom is NaN:', len(combined_gdf[combined_gdf['poly_geom'].isna()]))
print('img_geom_base is NaN:', len(combined_gdf[combined_gdf['img_geom_base'].isna()]))
print('img_geom_ev is NaN:', len(combined_gdf[combined_gdf['img_geom_ev'].isna()]))
print('img_geom_base OR img_geom_ev is NaN:', len(combined_gdf[(combined_gdf['img_geom_ev'].isna()) | combined_gdf['img_geom_base'].isna()]))
print('images for both baseline & EV:', len(combined_gdf[(~combined_gdf['img_geom_base'].isna()) & (~combined_gdf['img_geom_ev'].isna())]))

In [ ]:
# def reproject_geometry(geom, target_crs):
#     """
#     Reprojects a single Shapely geometry into the target CRS.

#     Args:
#         - geom (shapely geometry object): A geometry in the form of a shapely object. Expects an input CRS of EPSG:4326
#     Returns:
#         - A shapely object reprojected to the target_crs
#     """
#     gdf = gpd.GeoDataFrame(index=[0], geometry=[geom], crs="EPSG:4326")
#     return gdf.to_crs(target_crs).geometry.iloc[0]

In [ ]:
# def compute_shared_img_overlap(row, debug=False):
#     """
#     Given a row of a DataFrame with a polygon and baseline/early verification image footprints, 
#     calculates the actual area and % of the polygon area that has imagery coverage at both time points.

#     Args:
#         - row: Row in a GeoDataFrame
#     Returns:
#         - pd.Series with the area of the polygon with overlapping imagery (ha) and percent of the polygon with imagery coverage at both time points
#     """
#     from shapely.geometry.base import BaseGeometry

#     if debug:
#         print(f"\n⚙️ Processing poly_id: {row.get('poly_id', 'N/A')}")

#     # Extract row values
#     poly_geom = row['poly_geom']
#     img_geom_base = row['img_geom_base']
#     img_geom_ev = row['img_geom_ev']

#     if debug:
#         print(f"⬜ Geometry types:\n poly={type(poly_geom)},\n base={type(img_geom_base)},\n ev={type(img_geom_ev)}")


#     # Check for missing, empty, or invalid type geometries
#     #  If one or more geometries are missing/invalid, return 0 hectares of overlapping area and 0 % of the polygon with overlapping imagery coverage
#     if not all(isinstance(g, BaseGeometry) and not g.is_empty for g in [poly_geom, img_geom_base, img_geom_ev]):
#         if debug:
#             print(f"⚠️ Missing or invalid type of one or more geometries: recording 0 area & % overlap for poly_id: {row.get('poly_id')}")
#         return pd.Series({'overlap_area_both': 0, 'pct_img_cover_both': 0})
    
#     if debug:
#         print(f"✅ Valid geometries for polygon and both baseline & EV image")
    
#     # Compute UTM Zone based on polygon centroid
#     centroid = poly_geom.centroid
#     utm_crs = img_cover.get_utm_crs(centroid.x, centroid.y)
#     if debug:
#         print(f"🗺️ Calculated UTM CRS: {utm_crs}")

#     try:
#         # Reproject polygon and image geometries for accurate area calculations 
#         #  (Use reproject_geometry function to reproject shapely objects by wrapping them in a one-row GeoDataFrame)
#         poly_proj = reproject_geometry(poly_geom, utm_crs)
#         base_proj = reproject_geometry(img_geom_base, utm_crs)
#         ev_proj = reproject_geometry(img_geom_ev, utm_crs)
    
#         # Calculate intersection of polygon with baseline & early verification image
#         intersection = poly_proj.intersection(base_proj).intersection(ev_proj)
#         overlap_area = intersection.area
#         poly_area = poly_proj.area

#         # Calculate percent of polygon area with imagery at both baseline & early verification
#         percent_covered = (overlap_area / poly_area) if poly_area > 0 else 0

#         if debug:
#             print(f"📐 Polygon area (ha): {poly_area / 10_000}")
#             print(f"🛰️ Area with overlapping imagery (ha): {overlap_area / 10_000}")
#             print(f"✅ Percent of polygon with imagery coverage both years (ha): {percent_covered * 100}")

#         return pd.Series({
#             'overlap_area_both': overlap_area / 10_000,     # in hectares
#             'pct_img_cover_both': percent_covered * 100     # in % of polygon area
#             })
    
#     except Exception as e:
#         if debug:
#             print(f"❌ Exception occurred for poly_id {row.get('poly_id')}: {e}")
#         return pd.Series({'overlap_area_both': 0, 'pct_img_cover_both': 0})

### Apply Function to Calculate Area With Overlapping Imagery Across DataFrame

In [ ]:
combined_gdf[['overlap_area_both', 'pct_img_cover_both']] = combined_gdf.apply(
    analyze.compute_shared_img_overlap,
    axis=1
)

In [ ]:
combined_gdf[117:120]

In [ ]:
combined_gdf.to_csv('../data/BEF_shortlist/results/BEF_shortlist_poly_baseline_ev_img_cover_comparison.csv', index=False)

### Sanity-Checks of % of Polygon with Overlapping Imagery

In [ ]:
# Find suspicious polygons (100%) coverage in both years but 0% shared overlap
suspect_df = combined_gdf[
    (combined_gdf['pct_img_cover_base'] >= 99) &
    (combined_gdf['pct_img_cover_ev'] >= 99) &
    (combined_gdf['pct_img_cover_both'] == 0)
]

print(f"⚠️ Polygons with 100% base + EV coverage but 0% overlap: {len(suspect_df)}")
suspect_df[['poly_id', 'pct_img_cover_base', 'pct_img_cover_ev', 'pct_img_cover_both']].head()

In [ ]:
# Check for coverage in 'both' years but not in either year individually
overlap_but_missing_base_ev = combined_gdf[
    (combined_gdf['pct_img_cover_both'] > 0) &
    ((combined_gdf['pct_img_cover_base'] == 0) | (combined_gdf['pct_img_cover_ev'] == 0))
]

print(f"⚠️ Polygons with overlap but 0 coverage in base or EV: {len(overlap_but_missing_base_ev)}")

In [ ]:
# Summary statistics
combined_gdf[['pct_img_cover_base', 'pct_img_cover_ev', 'pct_img_cover_both']].describe()

In [ ]:
# Histograms
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

sns.histplot(combined_gdf['pct_img_cover_base'], bins=30, ax=axes[0], color='blue')
axes[0].set_title("Baseline Coverage (%)")

sns.histplot(combined_gdf['pct_img_cover_ev'], bins=30, ax=axes[1], color='green')
axes[1].set_title("Early Verification Coverage (%)")

sns.histplot(combined_gdf['pct_img_cover_both'], bins=30, ax=axes[2], color='purple')
axes[2].set_title("Overlap Coverage (%)")

plt.tight_layout()
plt.show()

In [ ]:
# Identify zones
no_coverage = combined_gdf[combined_gdf['pct_img_cover_both'] == 0]
print(f"📦 Polygons with 0% overlap coverage: {len(no_coverage)}")

coverage = combined_gdf[combined_gdf['pct_img_cover_both'] > 0]
print(f"📦 Polygons with overlapping coverage: {len(coverage)}")

len(no_coverage) + len(coverage)

In [ ]:
justdiggit = combined_gdf[combined_gdf['project_id'] == '8cbda7fc-aa17-444b-8425-7047fade708c']
jd_overlap = justdiggit[justdiggit['pct_img_cover_both'] > 0]
len(jd_overlap)


In [ ]:
print('Area of all polygons:', justdiggit['poly_area_ha'].sum())
print('Area of polygons with imagery at baseline:', justdiggit['overlap_area_base'].sum())
print('Area of polygons with imagery at EV:', justdiggit['overlap_area_ev'].sum())
print('Area of polygons with imagery at BOTH baseline & EV:', justdiggit['overlap_area_both'].sum())

## Test on single row

In [ ]:
test_row = combined_gdf.iloc[118]
analyze.compute_shared_img_overlap(test_row, debug=True)

In [ ]:
test_poly_geom = test_row['poly_geom']
test_img_geom_base = test_row['img_geom_base']
test_img_geom_ev = test_row['img_geom_ev']

In [ ]:
if test_img_geom_base is None:
    print('no img_base_geom!')
else:
    print(test_img_geom_base)

In [ ]:
centroid = test_poly_geom.centroid
utm_crs = img_cover.get_utm_crs(centroid.x, centroid.y)
utm_crs

In [ ]:
poly_proj = analyze.reproject_geometry(test_poly_geom, utm_crs)
base_proj = analyze.reproject_geometry(test_img_geom_base, utm_crs)
ev_proj = analyze.reproject_geometry(test_img_geom_ev, utm_crs)

In [ ]:
intersection = poly_proj.intersection(base_proj).intersection(ev_proj)

In [ ]:
print(intersection.geom_type)
print(intersection.is_valid)

In [ ]:
print(f"Intersection area (sq meters): {intersection.area}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Reprojected geometries (already in the same CRS)
gdf_poly = gpd.GeoDataFrame(geometry=[poly_proj], crs=utm_crs)
gdf_base = gpd.GeoDataFrame(geometry=[base_proj], crs=utm_crs)
gdf_ev   = gpd.GeoDataFrame(geometry=[ev_proj], crs=utm_crs)

#intersection = poly_proj.intersection(base_proj).intersection(ev_proj)

fig, ax = plt.subplots(figsize=(8,8))

# Plot the EV and baseline image footprints
gdf_base.plot(ax=ax, color='blue', alpha=0.5)
gdf_ev.plot(ax=ax, color='yellow', alpha=0.5)

# Plot the polygon outline
gdf_poly.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=2)

# Get polygon bounds (xmin, ymin, xmax, ymax)
xmin, ymin, xmax, ymax = gdf_poly.total_bounds

# Plot the true intersection of polygon, base image, & EV image
#if not intersection.is_empty:
#    gpd.GeoDataFrame(geometry=[intersection], crs=utm_crs).plot(ax=ax, color='red', alpha=0.5)

## Zoom to polygon extent
#ax.set_xlim([xmin, xmax])
#ax.set_ylim([ymin, ymax])
ax.set_aspect('equal')

# Custom legend
legend_elements = [
    mpatches.Patch(facecolor='blue', alpha=0.4, label='Baseline Image'),
    mpatches.Patch(facecolor='yellow', alpha=0.4, label='Early Verification Image'),
    mpatches.Patch(facecolor='green', alpha=0.4, label='Visual Overlap (Blue + Yellow)'),
    mpatches.Patch(facecolor='red', alpha=0.5, label='True Intersection'),
    mpatches.Patch(facecolor='none', edgecolor='black', label='Polygon')
]
ax.legend(handles=legend_elements, loc='upper right')

ax.set_title("Polygon and Image Footprints (Color Blended)")
plt.tight_layout()
plt.show()

### Aggregate the Area (& % Area) With Coverage at Both Years to the Project Level

In [ ]:
combined_gdf[117:120]

In [ ]:
type(combined_gdf.iloc[0]['pct_img_cover_both'])

In [ ]:
def aggregate_proj_dual_img_cover(combined_gdf, debug=False):
    """
    Aggregates polygon-level data to project level for imagery available at both baseline and early verification time points.

    Args:
        - combined_gdf (DataFrame): Polygon-level image coverage data. Requires the following columns:
            - 'poly_id' (str): unique ID for each polygon
            - 'project_id' (str): unique ID for each project
            - 'poly_area_ha' (float): the area of the polygon in hectares
            - 'overlap_area_both' (float): the area of the polygon that has imagery coverage at both baseline & early verification
            - 'pct_img_cover_both' (float): the percent of the polygon's overall area with imagery coverage at both time points
    Returns:
        - DataFrame: Project-level summary with total area, overlap area, and % coverage
    """

    # Group by project ID
    grouped = combined_gdf.groupby('project_id')

    if debug:
        print(f"🗂️ Analyzing {len(grouped)} projects")

    # Aggregate key metrics
    project_coverage_df = grouped.agg(
        num_polygons=('poly_id', 'count'),
        num_polygons_with_overlap=('pct_img_cover_both', lambda x: (x > 0).sum()),
        num_polygons_no_overlap=('pct_img_cover_both', lambda x: (x == 0).sum()),
        total_project_area_ha=('poly_area_ha', 'sum'),
        total_overlap_area_ha=('overlap_area_both', 'sum')
    ).reset_index()

    # Calculate % of total project area with overlapping coverage at both time points
    project_coverage_df['pct_project_img_cover_both'] = (
        (project_coverage_df['total_overlap_area_ha'] / project_coverage_df['total_project_area_ha']) * 100
    ).round(4)

    if debug:
        print("✅ Aggregation complete.")
    
    return project_coverage_df
   

In [ ]:
project_dual_coverage_df = aggregate_proj_dual_img_cover(combined_gdf, debug=True)

In [ ]:
approved_projects_df.head(2)

In [ ]:
# combined_gdf.merge(ev_img_avail_gdf[[
#     'poly_id', 'best_image', 'img_geom', 'overlap_area_ha', 'percent_img_cover']], how='left', on='poly_id').rename(
#     columns={
#         'best_image': 'best_img_ev',
#         'img_geom': 'img_geom_ev',
#         'overlap_area_ha': 'overlap_area_ev',
#         'percent_img_cover': 'pct_img_cover_ev'
#     })

# Merge in Country code
project_dual_coverage_df = project_dual_coverage_df.merge(approved_projects_df[['project_id', 'country']], how='left', on='project_id')

In [ ]:
project_dual_coverage_df.sort_values('pct_project_img_cover_both', ascending=False).head(10)

In [ ]:
print(len(project_dual_coverage_df[project_dual_coverage_df['pct_project_img_cover_both'] >= 70].sort_values('pct_project_img_cover_both', ascending=False)))
project_dual_coverage_df[project_dual_coverage_df['pct_project_img_cover_both'] >=70].sort_values('pct_project_img_cover_both', ascending=False)

In [ ]:
high_cov_proj_ids = project_dual_coverage_df[project_dual_coverage_df['pct_project_img_cover_both'] >=70].project_id.tolist()
high_cov_proj_ids

In [ ]:
combined_gdf_high_cov_projs = combined_gdf[combined_gdf['project_id'].isin(high_cov_proj_ids)]
print(len(combined_gdf_high_cov_projs))
print(len(combined_gdf_high_cov_projs['project_id'].unique()))
print(combined_gdf_high_cov_projs['project_id'].unique())
combined_gdf_high_cov_projs

combined_gdf_high_cov_projs.to_csv('../data/tf_cohort1/results/c1_poly_baseline_ev_img_cover_comparison_high_cov_projects.csv', index=False)

In [ ]:
project_dual_coverage_df.to_csv('../data/tf_cohort1/results/baseline_ev_img_cover_comparison.csv', index=False)

In [ ]:
project_dual_coverage_df_landscapes = project_dual_coverage_df[project_dual_coverage_df['country'] == 'CD']#.isin(['KE', 'GH', 'BI', 'RW', 'CD'])]
len(project_dual_coverage_df_landscapes)
project_dual_coverage_df_landscapes.sort_values('pct_project_img_cover_both', ascending=False)